In [149]:
import fasttext

"""
The training file should be this format:

__label__pp this is my text
__label__psoe this is also my text
"""

'\nThe training file should be this format:\n\n__label__pp this is my text\n__label__psoe this is also my text\n'

In [155]:
import pandas as pd
import numpy as np
import re
import emoji

#Function to get all emojis in a text.
def split_emojis(text):

    emoji_list = []
    data = re.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list

#Function to clean the text of a tweet.
def clean_tweet(text):
    #clean_text = re.sub("@.*:", "", text) #This do not improve performance.
    clean_text = re.sub("RT", "", text)
    #Separate emojis from words.
    """
    sep_emojis_text = ""
    for c in clean_text:
        if c in emoji.UNICODE_EMOJI:
            sep_emojis_text+=' '+c+' '
        else:
            sep_emojis_text+=c
     #This do not improve performance.
    """
    return clean_text

#Read and clean data for each party.

#Podemos.
up_cleaned_data = pd.DataFrame()
up_df = pd.read_csv ('up.csv')
up_cleaned_data["tweet"] = up_df["text"].apply(clean_tweet)
up_cleaned_data["label"] = "__label__up"
#Psoe.
psoe_cleaned_data = pd.DataFrame()
psoe_df = pd.read_csv ('psoe.csv')
psoe_cleaned_data["tweet"] = psoe_df["text"].apply(clean_tweet)
psoe_cleaned_data["label"] = "__label__psoe"
#CS.
cs_cleaned_data = pd.DataFrame()
cs_df = pd.read_csv ('cs.csv')
cs_cleaned_data["tweet"] = cs_df["text"].apply(clean_tweet)
cs_cleaned_data["label"] = "__label__cs"
#PP.
pp_cleaned_data = pd.DataFrame()
pp_df = pd.read_csv ('pp.csv')
pp_cleaned_data["tweet"] = pp_df["text"].apply(clean_tweet)
pp_cleaned_data["label"] = "__label__pp"
#Vox.
vox_cleaned_data = pd.DataFrame()
vox_df = pd.read_csv ('vox.csv')
vox_cleaned_data["tweet"] = vox_df["text"].apply(clean_tweet)
vox_cleaned_data["label"] = "__label__vox"

#Concat all parties data.
data = pd.concat([up_cleaned_data, psoe_cleaned_data, cs_cleaned_data, pp_cleaned_data, vox_cleaned_data])

display(data)

,tweet,label
0,"@JesusCintora: ""Sanitarios de la primaria de ...",__label__up
1,@DabitxoPiston: Donostia Scape Room,__label__up
2,@OskarMatute: Así se entiende mucho mejor el ...,__label__up
3,@IosuPosti: Hola! Mi nombre es Iosu Etayo Pal...,__label__up
4,"@OskarMatute: Pues, a riesgo de aburrir, lo r...",__label__up
...,...,...
4962,SE PODRA DECIR QUE EL ECHEMINGA ES UN DELINCUE...,__label__vox
4963,El colosal golpe de Herrera a Simancas por su ...,__label__vox
4964,"@MadamePerlan: @Macarena_Olona Hola Macarena,...",__label__vox
4965,@frandevox: Buenos días a todos desde la DlCT...,__label__vox


In [156]:
#Split data in training and test sets.
import csv
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data["tweet"], data["label"], test_size=0.20, random_state=42)

training = pd.concat([X_train,y_train], axis=1)
display(training)

test = pd.concat([X_test,y_test], axis=1)
display(test)

#Create training file.
training[training.columns.values[::-1]].to_csv('data.train', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

#Create test file.
test[test.columns.values[::-1]].to_csv('data.test', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

,tweet,label
3756,@AlmeidaPP_: Tenemos concepciones de la respo...,__label__pp
4107,@mmzfa 🤟,__label__vox
2325,"España es clave para China en Europa, es mejor...",__label__vox
4953,@JulianMaciasT: En este tuit está la captura ...,__label__up
3358,@euroefe: España será el primer país de la UE...,__label__psoe
...,...,...
654,@BeGoneSojanoi @electo_mania https://t.co/f9A4...,__label__vox
411,@Correos: Este año celebramos el #12deOctubre...,__label__psoe
860,@PabloIglesias: Esta mañana ha llegado el pri...,__label__up
5825,@InesArrimadas: Ofrecimos a PP y PSC una coal...,__label__cs


,tweet,label
1783,A punto de empezar el encuentro telemático con...,__label__cs
51,@Borja_AlvaCs: ☝️Ni una semana ha tardado la ...,__label__cs
1746,"@PSOE: 🗞️@Adrilastra en @eldiarioes \n\n""Son ...",__label__psoe
2203,@LeonDalitina5 @Clare1959 @Docaparicio Y como ...,__label__pp
3027,@UrolaKostaArt47 Gabon 🤗,__label__up
...,...,...
1521,@InesArrimadas: 😷 En junio logramos que el Co...,__label__cs
1629,@anadebande No nos parecemos en nada a VOX,__label__pp
308,"Teresa, no te han despedido. Sigues siendo dip...",__label__up
3786,El chiringuito del PP\n👇\n\nAyuso adjudica a d...,__label__psoe


In [160]:
import fasttext
# Load input training set.

#By default, fastText sees each training example only five times during training, which is pretty small. The number of times each examples is seen (also known as the number of epochs), can be increased using the -epoch option
model = fasttext.train_supervised(input="data.train", epoch=25)

In [161]:
#To predict one instance.
print( model.predict("Franco deberia volver a vivir") )
print( model.predict("Los españoles son los mejores 💙") )
print( model.predict("Los españoles son los mejores 💚") )
print( model.predict("España es una gran nacion") )
print( model.predict("Pablo Iglesias es el mejor") )
print( model.predict("Pablo Iglesias es un mierdas") )
print( model.predict("Putos progres de mierda") )
print( model.predict("Viva la guardia civil") )
print( model.predict("Igualdad salarial para las mujeres!") )
print( model.predict("La españa moderada") )
print( model.predict("La mujer a trabajar") )
print( model.predict("Viva la fuerza naranja") )

(('__label__vox',), array([0.99991167]))
(('__label__pp',), array([0.58533514]))
(('__label__vox',), array([0.99595493]))
(('__label__cs',), array([0.80102795]))
(('__label__up',), array([0.77848387]))
(('__label__up',), array([0.93949997]))
(('__label__vox',), array([1.00000513]))
(('__label__vox',), array([0.55404079]))
(('__label__up',), array([0.78779352]))
(('__label__psoe',), array([0.97950757]))
(('__label__pp',), array([0.84600508]))
(('__label__cs',), array([0.99322271]))


In [162]:
#To test the test set. (number of samples, precision, recall)
model.test("data.test")

precision = model.test("data.test")[1]
recall = model.test("data.test")[2]

F_score = 2*(precision * recall)/(precision + recall)

print("Precision:", precision)
print("Recall:", recall)
print("Fscore:", F_score)

Precision: 0.6925453843182696
Recall: 0.6925453843182696
Fscore: 0.6925453843182696
